## 微调任务主函数

In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from data_loading_preprocessing import load_data
from model import initialize_model
from training_fine_tuning import train_model, hyperparameter_tuning

c:\Users\heyh0\.conda\envs\deeplearning39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\heyh0\.conda\envs\deeplearning39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\heyh0\.conda\envs\deeplearning39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 设置设备
尽可能选择GPU进行训练

In [2]:
# 设置设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 数据加载
下载数据（如果数据已存在则不重复下载）, 生成批量数据

In [3]:
# 数据加载
download_url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'
train_loader, val_loader = load_data('.', batch_size=32, download_url=download_url)

# 创建包含训练和验证加载器的字典
dataloaders = {'train': train_loader, 'val': val_loader}

数据集已存在, 无需再次下载. 


# 超参数调优
对学习率和训练轮数进行调参（其它参数如动量和权重衰退是定值）

如果不想调参, 可以直接进行数据可视化, 那边提供了直接输入超参数的代码块


In [4]:
# 超参数列表
lr_list = [0.5e-2, 1e-2, 2e-2]
num_epochs_list = [20]

对预训练模型进行超参数调优

In [5]:
# 对预训练模型进行超参数调优
print("Tuning hyperparameters for pretrained model...")
model, _ = initialize_model("resnet", 200, use_pretrained=True)
model = model.to(device)
best_params_pretrained = hyperparameter_tuning(model, dataloaders, device, num_epochs_list, lr_list, use_pretrained=True)
print(f"Best hyperparameters for pretrained model: {best_params_pretrained}")

Tuning hyperparameters for pretrained model...
Training with lr=0.005, num_epochs=20
Final validation accuracy: 0.7164
New best accuracy: 0.7164 with lr=0.005 and num_epochs=20
Training with lr=0.01, num_epochs=20
Final validation accuracy: 0.7245
New best accuracy: 0.7245 with lr=0.01 and num_epochs=20
Training with lr=0.02, num_epochs=20
Final validation accuracy: 0.7370
New best accuracy: 0.7370 with lr=0.02 and num_epochs=20
Best Params: lr=0.02, num_epochs=20, Accuracy=0.7370
Best hyperparameters for pretrained model: {'lr': 0.02, 'num_epochs': 20, 'accuracy': tensor(0.7370, device='cuda:0', dtype=torch.float64)}


对随机初始化模型进行超参数调优

也可以不调优, 直接使用预训练模型的最优参数（一定意义上更公平）

In [ ]:
# 对随机初始化模型进行超参数调优
print("Tuning hyperparameters for model with random initialization...")
model, _ = initialize_model("resnet", 200, use_pretrained=False)
model = model.to(device)
best_params_random = hyperparameter_tuning(model, dataloaders, device, num_epochs_list, lr_list, use_pretrained=False)
print(f"Best hyperparameters for randomly initialized model: {best_params_random}")

# Tensorboard 可视化

In [6]:
# 准备TensorBoard
writer_pretrained = SummaryWriter('runs/pretrained_model')
writer_random = SummaryWriter('runs/random_init_model')

如果不想重新调参, 可以手动在下面输入最优参数

In [7]:
# 此代码是用来跳过调参, 直接运行的！
best_params_pretrained = {'lr': 0.02, 'num_epochs': 20}
best_params_random = {'lr': 0.02, 'num_epochs': 20}

重新训练预训练模型, 用Tensorboard可视化

In [8]:
# Re-train the pretrained model
model_pretrained, _ = initialize_model("resnet", 200, use_pretrained=True)
model_pretrained = model_pretrained.to(device)

# 设置不同的学习率
optimizer_pretrained = optim.SGD([
    {'params': model_pretrained.fc.parameters(), 'lr': best_params_pretrained['lr']},  # 最后一层
    {'params': (p for n, p in model_pretrained.named_parameters() if 'fc' not in n), 'lr': best_params_pretrained['lr'] / 10}  # 其他所有层
], momentum=0.9, weight_decay=1e-3)

criterion = nn.CrossEntropyLoss()
trained_model_pretrained = train_model(
    model_pretrained, dataloaders, device, criterion, optimizer_pretrained, num_epochs=best_params_pretrained['num_epochs'], writer=writer_pretrained
)
writer_pretrained.close()  # 关闭TensorBoard writer

# 保存模型的状态字典
torch.save(trained_model_pretrained.state_dict(), 'model_pretrained_state_dict.pth')

In [ ]:
# # 可以在这里直接导入模型的状态字典, 而不是重新训练, 可以更方便地进行进一步操作
# # 初始化模型
# trained_model_pretrained, _ = initialize_model("resnet", 200, use_pretrained=True)
# trained_model_pretrained = trained_model_pretrained.to(device)

# # 加载模型参数
# trained_model_pretrained.load_state_dict(torch.load('model_pretrained_state_dict.pth'))
# trained_model_pretrained.eval()  # 设置为评估模式

重新训练随机初始化模型, 用Tensorboard可视化

In [9]:
# Re-train the model from random initialization
model_random, _ = initialize_model("resnet", 200, use_pretrained=False)
model_random = model_random.to(device)

# 设置优化器
optimizer_random = optim.SGD(model_random.parameters(), lr=best_params_random['lr'], momentum=0.9, weight_decay=1e-3)

criterion = nn.CrossEntropyLoss()
trained_model_random = train_model(
    model_random, dataloaders, device, criterion, optimizer_random, num_epochs=best_params_random['num_epochs'], writer=writer_random
)
writer_random.close()  # 关闭TensorBoard writer

# 保存模型的状态字典
torch.save(trained_model_random.state_dict(), 'model_random_state_dict.pth')

In [ ]:
# # 可以在这里直接导入模型的状态字典, 而不是重新训练, 可以更方便地进行进一步操作
# # 初始化模型
# trained_model_random, _ = initialize_model("resnet", 200, use_pretrained=False)
# trained_model_random = trained_model_random.to(device)

# # 加载模型参数
# trained_model_random.load_state_dict(torch.load('model_random_state_dict.pth'))
# trained_model_random.eval()  # 设置为评估模式